# Selenium On Docker
This notebook does the following:
1. Spin up an exernal Selenium Docker container on the host.
2. Configures the remote Selenium Webdriver.
3. Sends commands to the Selenium Webdriver:
    We will be downloading mp3s from the bbc: wake up to money daily podcast.
4. Remove the container.

## Imports

In [48]:
import docker
import os
import time
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options

## Start Selenium container

In [49]:
cwd = os.getcwd()
local_downloads = '{}/downloads'.format(cwd)
sel_downloads = '/home/seluser/downloads'

In [50]:
client = docker.from_env()
container = client.containers.run('selenium/standalone-chrome',
        #command=['usermod', '-u 1000 {}'.format(sel_downloads)],
        #user='1000'
        volumes=['{}:{}'.format(local_downloads, sel_downloads),
                 '/dev/shm:/dev/shm'],
        ports={'4444/tcp':4444},
        network='container_bridge',
        detach=True)
cli = docker.APIClient()

## Configure webdriver

In [51]:
options = Options()
options.add_argument("--headless")
options.add_argument("--window-size=1920x1080")
chrome_driver = '{}:4444/wd/hub'.format('http://127.0.0.1') # This is only required for local development
#chrome_driver = '{}:4444/wd/hub'.format('http://ec2-52-31-168-109.eu-west-1.compute.amazonaws.com') # This is only required for local development

# wait for remote, unless timeout.
while True:
    try:
        driver = webdriver.Remote(
            command_executor=chrome_driver,
            desired_capabilities=DesiredCapabilities.CHROME, options=options)
        print('remote ready')
        break
    except:
        print('remote not ready, sleeping for ten seconds.')
        time.sleep(10)
        
# Enable downloads in headless chrome.
driver.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': sel_downloads}}
command_result = driver.execute("send_command", params)

remote not ready, sleeping for ten seconds.
remote ready


## Download Wake up to Money MP3 files
The configured webdriver will be used to download the mp3s from the BBC podcast Wake up to Money

In [70]:
from selenium_scripts.wake_up_to_money import download_podcast

In [71]:
download_podcast(driver,
                 'https://www.bbc.co.uk/programmes/b0070lr5/episodes/downloads',
                 local_downloads,
                 '20191011')

download started
file:  WakeUpToMoney-20191011-WakeUpToMoneyBusinessOfSportRugbyWorldCup.mp3.crdownload found
file still downloading, sleeping for 10 seconds
download complete


## Remove docker container

In [47]:
container.remove(force=True)
print('Removed container: {}'.format(container.id))

Removed container: dfa4913376d4a39db393bcb98334120f973b2d868215e864e15b7d501bdd38ac
